### Inverse Problems and Parameter Estimation, GEOS 627/427, University of Alaska Fairbanks

- script lab_pca.ipynb
- the goal of this tutorial is to establish/review the connections among eigen decomposition,
singular value decomposition, principal component analysis, covariance matrix, correlation matrix, etc.

- two key operations to the columns of the data matrix
(1) centered: subtract mean
(2) standardized (or scaled): divide by the standard deviation

In [ ]:
%matplotlib inline

In [ ]:
%run lib_header.py

In [ ]:
try:
    import scikit
except ImportError:
    import os
    os.system('pip install scikit-learn')

In [ ]:
import sys
sys.path.append('data/')
import pandas as pd
from pandas.plotting import scatter_matrix
from scipy import linalg as LA
from scipy import stats
from scipy.stats import zscore
from sklearn.decomposition import PCA

from lib_inverse import *                 # svdmat, pearsonr_ci
from load_pca_data import load_pca_data   # this resides in the ./data/ folder
from lib_geos import corrcov

In [ ]:
# Load data
# idata = 1: City ratings data (Matlab tutorial)
# idata = 2: Protein consumption data (GEOS 627 example)
idata = 1

X, dlabs, dlabslong, vlabs, vlabslong, ndata, nparm = load_pca_data(idata)

In [ ]:
#==========================================================================
### PART 1: MATLAB tutorial
# https://www.mathworks.com/help/stats/quality-of-life-in-u-s-cities.html

# Load data
# idata = 1: City ratings data
# idata = 2: Protein consumption data
idata = 1
X, dlabs, dlabslong, vlabs, vlabslong, ndata, nparm = load_pca_data(idata)

# Make a boxplot to look at the distribution of the protein consumption data
plt.figure(figsize=(10,8))
plt.boxplot(X,vert=False,labels=vlabslong)
plt.show()

In [ ]:
C = np.corrcoef(X,rowvar=False)
showmat(C,2)

In [ ]:
s = np.std(X,axis=0,ddof=1)
showmat([s],2)

In [ ]:
w = 1/s  # this differs from w in MATLAB code -- we implement weights manually 
W = np.tile(w,(ndata,1))
print(X.shape)
print(W.shape)
#showmat(W,2)
Xprime = X*W    # this is NOT matrix multiplication

In [ ]:
pca = PCA()
pca.fit(Xprime)
coefforth = pca.components_.T  # 
wcoeff = (np.tile(s.flatten(),(nparm,1))*LA.inv(coefforth)).T  # wcoeff from MATLAB weighted PCA
score = pca.fit_transform(Xprime)  # score
latent = np.reshape(pca.explained_variance_, (nparm,1))  # latent
cscores = zscore(X,ddof=1)@coefforth
# how to get tsquared?
explained = pca.explained_variance_ratio_ * 100

In [ ]:
# visualizing the results
plt.figure()
plt.plot(score[:,0],score[:,1],'+')
plt.xlabel('1st Principal Component')
plt.ylabel('2nd Principal Component')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(range(1,nparm+1),explained, color='C0')
ax.set_xlim([0,10])
ax.set_xticks(range(1,nparm+1))
ax.tick_params(axis='y', colors='C0')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Variance Explained (%)',color='C0')
ax2 = ax.twinx()
ax2.plot(range(0,nparm+1),[0, *list(np.cumsum(explained))], color="C3", marker="D", ms=7)
ax2.set_ylim([0,110])
ax2.tick_params(axis='y', colors='C3')
ax2.set_ylabel('Cumulative Percentage (%)', color='C3')
plt.show()

In [ ]:
# Create biplot function similar to MATLAB's
def create_biplot(score,coeff,labels=None):
    plt.figure(figsize=(10,8))
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.axvline(0,color='k',alpha=0.5)
    plt.axhline(0,color='k',alpha=0.5)
    plt.scatter(xs * scalex,ys * scaley,s=7, color='red')
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1], color='blue', alpha=0.8)
        if labels is None:
            plt.text(coeff[i,0]*1.05, coeff[i,1]*1.05, "Var"+str(i+1), color='k', ha='center', va='center')
        else:
            plt.text(coeff[i,0]*1.05, coeff[i,1]*1.05, labels[i], color='k', ha='center', va='center')
    plt.xlabel("Principal Component {}".format(1))
    plt.ylabel("Principal Component{}".format(2))
    plt.grid()

# Create biplot for first two principal components
create_biplot(score[:,0:2],np.transpose(coefforth.T[0:2, :]),vlabslong)
plt.xlim([-0.55,0.55])
plt.ylim([-0.55,0.55])
plt.show()

In [ ]:
#==========================================================================
### PART 2

# Load data
# idata = 1: City ratings data
# idata = 2: Protein consumption data
idata = 2
X, dlabs, dlabslong, vlabs, vlabslong, ndata, nparm = load_pca_data(idata)

# Make a boxplot to look at the distribution of the protein consumption data
plt.figure(figsize=(10,8))
plt.boxplot(X,vert=False,labels=vlabslong)
plt.show()

In [ ]:
n,p = np.shape(X)
xinds = range(1,nparm+1)
X0 = X

# correlation coefficients
R = np.corrcoef(X0,rowvar=False)
R2 = np.triu(R)
Matrix(R2)

# display correlation coefficients '
print(''.join(['%10s',*['%8s' for i in range(p)]]) % ('',*vlabs))
for ii in range(p):
    print(''.join(['%10s',*['%8.3f' for i in range(p)]]) % (vlabslong[ii],*R2[ii,:]))
    
print('\nlargest correlation coefficient magnitudes:')
Rtemp = R2
Rtemp[np.where(Rtemp>0.999)] = np.nan
Matrix(Rtemp)
for kk in range(20):
    val = np.nanmax(abs(Rtemp))
    ii,jj = np.unravel_index(np.nanargmax(abs(Rtemp)), Rtemp.shape)
    # _,p = stats.pearsonr(X[:,ii],X[:,jj])
    _,pval,rlo,rup = pearsonr_ci(X[:,ii],X[:,jj],alpha=0.05)
    print('  corr[%10s,%10s] = %6.3f  [%6.3f, %6.3f], p = %6.3f' % 
          (vlabslong[ii],vlabslong[jj],Rtemp[ii,jj],rlo,rup,pval))
    Rtemp[ii,jj] = np.nan

In [ ]:
# scatterplot matrix: uncentered, unstandardized
df = pd.DataFrame(X0,columns=vlabslong)
scatter_matrix(df, alpha=1, figsize=(8,8), diagonal='hist')
plt.show()

In [ ]:
# Exploring relationships among eig, cov, svd, pca, etc
# from wiki: http://en.wikipedia.org/wiki/Principal_component_analysis

In [ ]:
### REMOVE THE MEAN FOR EACH COLUMN
u = np.mean(X,axis=0)         # dimensionless vector
print(vlabslong)
showmat([u],2)
u = np.reshape(u,(1,len(u)))  # row vector
ones = np.ones((ndata,1))        # column vector
print(ones.shape)
print(u.shape)
print((ones@u).shape)
#showmat(ones@u,2)
# Craft the centered matrix
B = X - ones@u
#showmat(B,2)

In [ ]:
### COVARIANCE MATRIX
C = (1/(ndata-1)) * (B.T @ B)  
# LA.norm(C-np.cov(B,rowvar=False)) / LA.norm(C) # check
# LA.norm(C-np.cov(X,rowvar=False)) / LA.norm(C) # check

# standard deviations of each column of X
s = np.sqrt(np.diag(C))        # dimensionless vector
s = np.reshape(s, (1,len(s)))  # row vector
# s = np.sqrt(np.var(X, axis=0, ddof=1))) # alternative
showmat(s,2)

# important: np.sqrt and np.var differ from MATLAB's sqrt and var 
# due to default setting of ddof=0

# check for sigma1
Cdiag = np.diag(np.diag(C))
hCdiag = LA.sqrtm(Cdiag)
# print([np.std(X[:,0], ddof=1), hCdiag[0][0]]) # check
# print([np.var(X[:,0], ddof=1), Cdiag[0][0]]) # check

In [ ]:
### CORRELATION MATRIX (is a scaled covariance matrix)
# R = np.corrcoef(X, rowvar=False) # alternative
# R = np.corrcoef(B, rowvar=False) # alternative
# R = corrcov(C) # alternative
# R = C/(s.T @ s) # alternative
# R = np.diag(1/s.flatten()) @ C @ np.diag(1/s.flatten()) # alternative
R = LA.inv(hCdiag) @ C @ LA.inv(hCdiag)
# Ccheck = hCdiag @ R @ hCdiag # check
# LA.norm(Ccheck - C) / LA.norm(C) # check

In [ ]:
### STANDARDIZED (AND CENTERED) MATRIX
# Z = B / (h@s) # alternative
# Z = B @ np.diag(1/s.flatten()) # alternative
Z = B @ LA.inv(hCdiag)
# LA.norm(Z - zscore(X,ddof=1)) / LA.norm(Z) # check
# Bcheck = Z * (h@s) # check reverse
# LA.norm(B-Bcheck) / LA.norm(B) # check reverse

In [ ]:
### EIGENVALUE DECOMPOSITION
eigvalC, Vc = LA.eig(C)
Dc = np.real(np.diag(eigvalC))
# LA.norm(C@Vc - Vc@Dc) / LA.norm(C@Vc) # check
## if needed: sort eigenvalues and rearrange V
isort = np.argsort(eigvalC)[::-1] # [::-1] flips the array
Vc = Vc[:,isort]
Dc = np.diag(eigvalC[isort])
# LA.norm(C@Vc - Vc@Dc) / LA.norm(C@Vc) # check
eigvalC = eigvalC[isort]  # dimensionless
eigvalC = np.reshape(eigvalC,(nparm,1))  # column vector
showmat(eigvalC.T,2)

eigvalR, Vr = LA.eig(R)
Dr = np.real(np.diag(eigvalR))
isort = np.argsort(eigvalR)[::-1] # [::-1] flips the array
Vr = Vr[:,isort]
Dr = np.diag(eigvalR[isort])
# LA.norm(R@Vr - Vr@Dr) / LA.norm(R@Vr) # check
eigvalR = eigvalR[isort]  # dimensionless
eigvalR = np.reshape(eigvalR,(nparm,1))  # column vector
showmat(eigvalR.T,2)
# Vr @ Dr @ Vr.T
# LA.inv(hCdiag) @ Vc @ Dc @ (LA.inv(hCdiag) @ Vc).T

In [ ]:
experiment = False

if experiment:
    # experiment with multiplying each column of A by a factor
    A = np.random.rand(8,3)
    s = np.sqrt(np.var(A, axis=0, ddof=1))  # dimensionless
    s = np.reshape(s,(1,3))  # row vector
    h = np.ones((8,1))  # column vector
    print('A = \n', A)
    # this looks like dividing each column by its standard deviation
    ans = A / (h@s)
    print('A / (h@s) = \n', ans)
    ans = A / np.tile(s,(8,1))
    print('A / np.tile(s,(8,1)) = \n', ans)
    # this is the series of elemetary matrix operations
    ans = A @ np.diag([1/s[0,0],1,1]) @ np.diag([1,1/s[0,1],1]) @ np.diag([1,1,1/s[0,2]])
    # this is what it means
    ans = A @ np.diag(1/s.flatten())
    print('A @ np.diag(1/s.flatten()) = \n', ans)

In [ ]:
print('mean(X) =')
showmat([np.mean(X,axis=0)],2)
print('mean(B) =')
showmat([np.mean(B,axis=0)],2)
print('mean(Z) =')
showmat([np.mean(Z,axis=0)],2)
print('var(X) =')
showmat([np.var(X,axis=0,ddof=1)],2)
print('var(B) =')
showmat([np.var(B,axis=0,ddof=1)],2)
print('var(Z) =')
showmat([np.var(Z,axis=0,ddof=1)],2)
print('std(X) =')
showmat([np.std(X,axis=0,ddof=1)],2)
print('std(B) =')
showmat([np.std(B,axis=0,ddof=1)],2)
print('std(Z) =')
showmat([np.std(Z,axis=0,ddof=1)],2)

In [ ]:
### SINGULAR VALUE DECOMPOSITION of B
# the scores matrix is nothing more than U@S from the SVD
Ub,Sb,Vb = svdmat(B)
svalb = np.diag(Sb)  # dimensionless
svalb = np.reshape(svalb,(nparm,1))  # column vector
USb = Ub@Sb
# check singular values with eigenvalues of covariance matrix
print(LA.norm(np.square(svalb)/(ndata-1) - eigvalC) / LA.norm(eigvalC))
# check: Vb = Vc. Use abs() to allow for sign flips
# abs(Vc) - abs(Vb)

### SINGULAR VALUE DECOMPOSITION of Z
# Vz = Vr (allowing sign changes)
Uz,Sz,Vz = svdmat(Z)
svalz = np.diag(Sz)  # dimensionless
svalz = np.reshape(svalz,(nparm,1))  # column vector
USz = Uz@Sz
# check singular values with eigenvalues of correlation matrix
print(LA.norm(np.square(svalz)/(ndata-1) - eigvalR) / LA.norm(eigvalR))

In [ ]:
### Test 1: Use centered matrix
# VB = Vb = Vc (allowing for some sign flips on columns of V)
# USB = Ub@Sb
# pcvarB = eigvalC
pca = PCA()
pca.fit(B)
VB = pca.components_.T  # coeff
USB = pca.fit_transform(B)  # score
pcvarB = np.reshape(pca.explained_variance_, (nparm,1))  # latent
showmat(pcvarB.T,2)
showmat(eigvalC.T,2)
# this is equivalent, since pca will center the matrix (i.e., remove mean)
# pca = PCA()
# pca.fit(X)
# V = pca.components_.T  # coeff
# US = pca.fit_transform(X)  # score

# Bcheck = USB @ VB.T
# LA.norm(B-Bcheck) / LA.norm(B)
# # orthonormal
# LA.norm(VB.T @ VB - np.eye(nparm))
# US1_check = B @ VB
# LA.norm(USB - US1_check) / LA.norm(USB)

In [ ]:
### Test 2 (example used in matlab)
# note: this gives different US and V from Test 1
w = 1/s  # this differs from w in MATLAB code -- we implement weights manually
W = np.tile(w,(ndata,1))
Bprime = B*W
pca = PCA()
pca.fit(Bprime)
Vw_not_orthonormal = pca.components_.T  # 
Vw = (np.tile(s.flatten(),(nparm,1))*LA.inv(Vw_not_orthonormal)).T  # wcoeff from MATLAB weighted PCA
USw = pca.fit_transform(Bprime)  # score
pcvarBw = np.reshape(pca.explained_variance_, (nparm,1))  # latent

# % this is equivalent
# %[Vw,USw] = pca(B,'VariableWeights','variance')
# Bcheck = USw * Vw';
# norm(B - Bcheck) / norm(B)
# disp('Vw is NOT orthornomal:');
# norm(Vw'*Vw - eye(nparm))
# Vworth = inv(hCdiag)*Vw;   % note: inv(hCdiag) = diag(1./s)
# % orthonormal:
# norm(Vworth'*Vworth - eye(nparm))
# % this shows how USw can be computed
# USw_check = Z*Vworth;
# norm(USw - USw_check) / norm(USw)

In [ ]:
### Test 3: Use centered+standardized matrix as input
# VZ = Vz = Vr = LA.inv(hCdiag)@Vw  (allowing for sign flips)
# USZ = USw (allowing for sign flips)
# pcvarZ = eigvalR
pca = PCA()
pca.fit(Z)
VZ = pca.components_.T  # coeff
USZ = pca.fit_transform(Z)  # score
pcvarZ = np.reshape(pca.explained_variance_, (nparm,1))  # latent

# Zcheck = USZ @ VZ.T
# LA.norm(Z-Zcheck) / LA.norm(Z)
# Bcheck = USZ @ VZ.T @ hCdiag
# LA.norm(B-Bcheck) / LA.norm(B)
# # orthonormal
# LA.norm(VZ.T @ VZ - np.eye(nparm))
# VZ_check = LA.inv(hCdiag)@Vw
# LA.norm(VZ - VZ_check) / VZ.norm(USB)

In [ ]:
#==============================
# tips for the lab exercise (lab_pca.m)

# cumulative variance
for kk in range(2):
    if kk == 0:
        pcvar = pcvarB
        stlab = 'centered'
    else:
        pcvar = pcvarZ
        stlab = 'centered+standardized'
    propvar = pcvar/np.sum(pcvar)
    # from the matlab tutorial: explained = 100*cpropvar (idata=1)
    cpropvar = np.cumsum(propvar)
    print('\n')
    print('Importance of principal components [%s]' % stlab)
    print('Std-Dev  : sqrt( lam_k )')
    print('Var      : lam_k = s_k^2 / (ndata-1) : eigenvalues of the covariance (or correlation) matrix of X')
    print('Prop-Var : proportion of variance')
    print('Std-Dev  : cumulative proportion of variance')
    print('----------------------------------------------')
    print('PC#\tStd-Dev\t\tVar\t\tProp-Var\tCum-Prop')
    for i in range(nparm):
        print('%d\t%f\t%f\t%f\t%f' % (i+1,np.sqrt(pcvar[i]),pcvar[i],propvar[i],cpropvar[i]))

In [ ]:
# plot components
fig=plt.figure(figsize=(10,14))
for ii in range(nparm):
    plt.subplot(5,2,ii+1)
    plt.plot([0.5,nparm+0.5],[0,0],'k')
    h1 = plt.plot(range(1,nparm+1),VB[:,ii],'r.-',label='centered')
    h2 = plt.plot(range(1,nparm+1),VZ[:,ii],'b.-',label='standardized')
    plt.axis([0,nparm+1,-1,1])
    plt.title('PC-%d' % (ii+1))
    plt.xticks(range(1,nparm+1),labels=vlabs)
    plt.grid()
    if ii == 0:
        plt.legend()
        plt.subplots_adjust(wspace=0.2,hspace=0.5)